
## 05 Hyperparameter Tuning
**Objetivo**: Optimizar los hiperparámetros del modelo seleccionado.
**Contenido**:
- Uso de técnicas como Grid Search y Random Search.
- Evaluación del rendimiento del modelo optimizado.


In [1]:
## Importar librerías
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import xgboost as xgb

## Cargar dataset de entranamiento y evaluación

In [22]:

dir_data_processed = '../data/processed'
ruta_archivo_X = os.path.join(dir_data_processed, 'X_train_resampled.csv')
X_train_resampled = pd.read_csv(ruta_archivo_X)

ruta_archivo_y = os.path.join(dir_data_processed, 'y_train_resampled.csv')
y_train_resampled = pd.read_csv(ruta_archivo_y)

ruta_archivo_X = os.path.join(dir_data_processed, 'X_val.csv')
X_val = pd.read_csv(ruta_archivo_X)

ruta_archivo_y = os.path.join(dir_data_processed, 'y_val.csv')
y_val = pd.read_csv(ruta_archivo_y)


In [3]:
X_train_resampled.shape, y_train_resampled.shape

((14604, 22), (14604, 1))

In [4]:
X_val.shape, y_val.shape

((64900, 22), (64900, 1))

## Definir el modelo base

In [5]:

rf = RandomForestClassifier(random_state=42)
y_train_resampled = y_train_resampled.values.ravel()
y_val = y_val.values.ravel()

##  Definir los parámetros para Grid Search y Random Search

In [6]:
# Definir los parámetros que se van a probar en Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],  # Número de árboles en el bosque
    'max_features': ['sqrt', 'log2'],  # Número máximo de features consideradas para dividir un nodo
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],  # Número mínimo de muestras requeridas para dividir un nodo interno
    'min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas para ser una hoja
    'bootstrap': [True, False]  # Método de muestreo de instancias para construir árboles
}

# Inicializar Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)


In [7]:
# Definir la distribución de parámetros para Random Search
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Inicializar Random Search
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42)


## Entrenar y ajustar hiperparámetros

Combinamos Grid Search y Random Search para explorar de manera exhaustiva y eficiente el espacio de hiperparámetros. Utilizamos Grid Search inicialmente para realizar una búsqueda amplia de hiperparámetros, seguida de Random Search para refinar y buscar de manera más precisa dentro de las regiones prometedoras identificadas.

### Grid Search

In [8]:
# grid_search.fit(X_train_resampled, y_train_resampled)
# print("Mejores hiperparámetros encontrados por Grid Search:")
# print(grid_search.best_params_)


### Random Search

In [9]:
# Definir la distribución de parámetros para Random Search
param_dist = {
    'n_estimators': [300],
    'max_features': ['sqrt'],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [2],
    'bootstrap': [True]
}

# Inicializar Random Search
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42)


In [10]:
random_search.fit(X_train_resampled, y_train_resampled)
print("Mejores hiperparámetros encontrados por Random Search:")
print(random_search.best_params_)


/home/joseph/Documents/proyecto-portafolio/deteccion_fraude/venv/lib/python3.11/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 1 is smaller than n_iter=100. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores hiperparámetros encontrados por Random Search:
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}


## Evaluar el modelo optimizado

In [11]:
# Función para evaluar el rendimiento del modelo
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:")
    roc_auc = roc_auc_score(y_val, y_pred)
    print(f'Área bajo la curva ROC (ROC AUC): {roc_auc:.2f}')
    print(classification_report(y_test, y_pred))

# Evaluación del modelo con los mejores parámetros encontrados por Grid Search
# best_grid_model = grid_search.best_estimator_
# print("Rendimiento del modelo con mejores parámetros (Grid Search):")
# evaluate_model(best_grid_model, X_val, y_val)

# Evaluación del modelo con los mejores parámetros encontrados por Random Search
best_random_model = random_search.best_estimator_
print("Rendimiento del modelo con mejores parámetros (Random Search):")
evaluate_model(best_random_model, X_val, y_val)

Rendimiento del modelo con mejores parámetros (Random Search):
Accuracy: 0.7750847457627119
Classification Report:
Área bajo la curva ROC (ROC AUC): 0.69
              precision    recall  f1-score   support

           0       0.99      0.78      0.87     63324
           1       0.06      0.61      0.12      1576

    accuracy                           0.78     64900
   macro avg       0.53      0.69      0.49     64900
weighted avg       0.97      0.78      0.85     64900



x_

In [12]:
# Crear un clasificador XGBoost
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

In [13]:
# # Definir los parámetros que se van a probar en Grid Search
# param_grid = {
#     'n_estimators': [100, 200, 300],  # Número de árboles (boosting rounds)
#     'max_depth': [3, 4, 5],  # Profundidad máxima del árbol
#     'learning_rate': [0.1, 0.01, 0.05],  # Tasa de aprendizaje
#     'subsample': [0.8, 0.9, 1.0],  # Proporción de muestras utilizadas para entrenar cada árbol
#     'colsample_bytree': [0.8, 0.9, 1.0],  # Proporción de características utilizadas para entrenar cada árbol
#     'gamma': [0, 1, 5]  # Reducción de la pérdida mínima requerida para dividir un nodo
# }

# # Inicializar Grid Search
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)


{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}

## Utilizando Xgboost como segunda propuesta de modelo

In [14]:
param_dist = {
    'max_depth': [ 10],  # Profundidad máxima del árbol
    'eta': [ 0.3],     # Tasa de aprendizaje (learning rate)
    'objective': ['binary:logistic'],      # Función objetivo (clasificación binaria)
    'eval_metric': ['logloss', 'auc']      # Métricas de evaluación
    
}


random_search = RandomizedSearchCV(
    estimator=xgb_model,     # Modelo XGBoost
    param_distributions=param_dist,  # Distribución de parámetros para explorar
    n_iter=100,              # Número de combinaciones de hiperparámetros a probar
    cv=5,                    # Número de divisiones para validación cruzada
    scoring='roc_auc',       # Métrica de evaluación a optimizar (AUC-ROC)
    n_jobs=-1,               # Número de núcleos para procesos en paralelo, -1 usa todos los disponibles
    random_state=42          # Semilla para reproducibilidad de resultados
)

In [15]:
# grid_search.fit(X_train_resampled, y_train_resampled)
# print("Mejores hiperparámetros encontrados por Grid Search:")
# print(grid_search.best_params_)


In [16]:
random_search.fit(X_train_resampled, y_train_resampled)
print("Mejores hiperparámetros encontrados por Random Search:")
print(random_search.best_params_)


/home/joseph/Documents/proyecto-portafolio/deteccion_fraude/venv/lib/python3.11/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=100. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores hiperparámetros encontrados por Random Search:
{'objective': 'binary:logistic', 'max_depth': 10, 'eval_metric': 'logloss', 'eta': 0.3}


In [17]:


# # Evaluación del modelo con los mejores parámetros encontrados por Grid Search
# best_grid_model = grid_search.best_estimator_
# print("Rendimiento del modelo con mejores parámetros (Grid Search):")
# evaluate_model(best_grid_model, X_test, y_test)

# Evaluación del modelo con los mejores parámetros encontrados por Random Search
best_random_model = random_search.best_estimator_
print("Rendimiento del modelo con mejores parámetros (Random Search):")
evaluate_model(best_random_model, X_val, y_val)


Rendimiento del modelo con mejores parámetros (Random Search):
Accuracy: 0.7363174114021571
Classification Report:
Área bajo la curva ROC (ROC AUC): 0.75
              precision    recall  f1-score   support

           0       0.99      0.74      0.84     63324
           1       0.07      0.77      0.12      1576

    accuracy                           0.74     64900
   macro avg       0.53      0.75      0.48     64900
weighted avg       0.97      0.74      0.83     64900



In [18]:
dir_data_processed = '../models'
ruta_archivo_model = os.path.join(dir_data_processed, 'modelo_xgboost.bin')

best_random_model.save_model(ruta_archivo_model )

/home/joseph/Documents/proyecto-portafolio/deteccion_fraude/venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:54:41] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


## **Conlusión**

- No se observó una mejora significativa con la optimización en Random Forest, pero sí se encontró una ligera mejora con el modelo XGBoost, aunque sigue siendo marginal. Se sugiere considerar agregar o reemplazar la columna de TransactionAmt como una posible mejora para el entrenamiento del modelo.


- No se observó una mejora significativa con la optimización en Random Forest, pero sí se encontró una ligera mejora con el modelo XGBoost, aunque sigue siendo marginal. Se sugiere considerar agregar o reemplazar la columna de TransactionAmt como una posible mejora para el entrenamiento del modelo.